In [ ]:
path = "c:/Users/Nathalie/Nextcloud/LADi/Orthografie Trainer/Code/04_bias_mitigation/"

import sys

sys.path.insert(1, path + "classes")

from DecisionTree_insession import in_session_decision_tree
from KNN_insession import in_session_KNN
from MLP_insession import in_session_MLP
from MetricsClass import Evaluation
from Plotting import Plots

import pandas as pd
import seaborn as sns
import pickle

Aggregation Bias Mitigation

Group 1: Parents with high school diploma (ABI)

In [ ]:
# define feature cols
feature_cols = [
    "Erstloesung",
    "Schussel",
    "Erfolg",
    "Schwierigkeit",
    "ist_Schulzeit",
    "MehrfachFalsch",
    "vorher_abgebrochen",
    "Fehler",
    "Klassenstufe",
    "Jahredabei",
    "Testposition__pruefung",
    "Testposition__training",
    "Testposition__version",
    "Art__GK",
    "Art__GR",
    "Art__GZ",
    "Art__K",
    "Art__LB",
    "UserAttribut",
    "OrderNumber",
    "steps",
]

Decision Tree Classifier

In [ ]:
DTE = in_session_decision_tree()
DTE.set_range(2, 61)
DTE.set_majority_group("AbiEltern")
DTE.set_demographic_category("AbiEltern")

DTE.load_matrices(
    "05_aggregation_bias_mitigation/parental_education/matrix_abi", ".pkl", False
)

DTE.set_feature_cols(feature_cols)

DTE.set_majority_group("abi")

metrics_DTE = DTE.loop_matrices("parental_education_representation/matrix_abi", 5, 1, 2)

KNN

In [ ]:
KNN = in_session_KNN()
KNN.set_range(2, 61)

KNN.set_majority_group("AbiEltern")
KNN.set_demographic_category("AbiEltern")

KNN.load_matrices(
    "05_aggregation_bias_mitigation/parental_education/matrix_abi", ".pkl", False
)
KNN.set_feature_cols(feature_cols)
KNN.set_majority_group("abi")

metrics_KNN = KNN.loop_matrices("parental_education/matrix_abi", 2, "uniform")

MLP

In [ ]:
MLP = in_session_MLP()
MLP.set_range(2, 61)

MLP.set_majority_group("AbiEltern")
MLP.set_demographic_category("AbiEltern")

MLP.load_matrices(
    "05_aggregation_bias_mitigation/parental_education/matrix_abi", ".pkl", False
)
MLP.set_feature_cols(feature_cols)
MLP.set_majority_group("abi")
metrics_MLP = MLP.loop_matrices(
    "parental_education/matrix_abi", "Adam", "binary_crossentropy", "accuracy", 21, 21
)

Group 2: keinAbi

Decision Tree Classifier

In [ ]:
DTE = in_session_decision_tree()
DTE.set_range(2, 61)
DTE.set_majority_group("AbiEltern")
DTE.set_demographic_category("AbiEltern")

DTE.load_matrices(
    "05_aggregation_bias_mitigation/parental_education/matrix_keinAbi", ".pkl", False
)

DTE.set_feature_cols(feature_cols)

DTE.set_majority_group("keinAbi")

metrics_DTE = DTE.loop_matrices(
    "parental_education_representation/matrix_keinAbi", 5, 1, 2
)

KNN

In [ ]:
KNN = in_session_KNN()
KNN.set_range(2, 61)

KNN.set_majority_group("AbiEltern")
KNN.set_demographic_category("AbiEltern")

KNN.load_matrices(
    "05_aggregation_bias_mitigation/parental_education/matrix_keinAbi", ".pkl", False
)
KNN.set_feature_cols(feature_cols)
KNN.set_majority_group("keinAbi")

metrics_KNN = KNN.loop_matrices("parental_education/matrix_keinAbi", 2, "uniform")

MLP

In [ ]:
MLP = in_session_MLP()
MLP.set_range(2, 61)

MLP.set_majority_group("AbiEltern")
MLP.set_demographic_category("AbiEltern")

MLP.load_matrices(
    "05_aggregation_bias_mitigation/parental_education/matrix_keinAbi", ".pkl", False
)
MLP.set_feature_cols(feature_cols)
MLP.set_majority_group("keinAbi")
metrics_MLP = MLP.loop_matrices(
    "parental_education/matrix_keinAbi",
    "Adam",
    "binary_crossentropy",
    "accuracy",
    21,
    21,
)

Model metrics

In [ ]:
metrics = pd.concat([metrics_DTE, metrics_KNN, metrics_MLP])

In [ ]:
## plot count of sentences by n and subgroup
ax = sns.lineplot(data=metrics, x="Sentence", y="Length", hue="subgroup")

In [ ]:
## construct dfs for both groups from metric df
grouped = metrics.groupby(metrics.subgroup)
df_abi = grouped.get_group("abi")
df_keinAbi = grouped.get_group("keinAbi")

# clean wording
df_abi["model"] = df_abi["model"].replace(["DL"], "MLP")
df_keinAbi["model"] = df_keinAbi["model"].replace(["DL"], "MLP")

In [ ]:
## plot accuracy by n and model
ax = sns.lineplot(data=df_abi, x="Sentence", y="Accuracy", hue="model")

In [ ]:
## plot accuracy by n and model
ax = sns.lineplot(data=df_keinAbi, x="Sentence", y="Accuracy", hue="model")

Evaluation

In [ ]:
# evaluate metrics for fairness
evaluation_obj = Evaluation(metrics)
evaluation_obj.set_demographic_category("AbiEltern")
evaluation_obj.set_minority_group("abi")
evaluation_obj.set_majority_group("keinAbi")
df_AbiEltern = evaluation_obj.evaluate()

In [ ]:
# show table
table = evaluation_obj.showTable(df_AbiEltern)
table

In [ ]:
# save
writer = pd.ExcelWriter("df_parental_edu_AggBias.xlsx", engine="xlsxwriter")
df_AbiEltern.to_excel(writer, sheet_name="parental_edu")
writer.save()